In [56]:
import plotly_express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
import os

In [4]:
PRICES_PATH = "../../data_processed/resale_flat_prices/engineered_data.csv"
original_df = pd.read_csv(PRICES_PATH)
pred_df = pd.read_csv("xgboost_pred_table.csv")

In [5]:
display(original_df.head(10))
display(pred_df.head(10))

,id,town,flat_type,flat_model,sale_date,floor,age,relative_tenure,floor_area,psm,...,nearest_post_offices,nearest_bus_stops,nearest_pharmacies,nearest_primary_schools,nearest_parks,nearest_trains,sale_month,cpi,cpi_psm,avg_sora
0,1,JURONG_WEST,3_ROOM,IMPROVED,1990-01-01,5.0,16,0.899953,70.0,334.285714,...,0.798752,0.195397,0.761661,0.541789,0.947906,0.904104,1,61.9,595.125779,NaN
1,2,OTHER,5_ROOM,STANDARD,1990-01-01,5.0,15,0.902280,120.0,916.666667,...,3.042559,0.090376,1.133326,0.159056,0.330270,1.954674,1,61.9,1631.933226,NaN
2,3,BUKIT_MERAH,3_ROOM,IMPROVED,1990-01-01,5.0,15,0.902280,63.0,722.222222,...,1.148847,0.446105,0.850333,0.943485,0.737703,0.932695,1,61.9,1285.765572,NaN
3,4,BUKIT_MERAH,OTHER,IMPROVED,1990-01-01,8.0,21,0.887322,29.0,251.724138,...,1.294327,1.620819,0.431442,0.691163,0.311404,0.448964,1,61.9,448.142165,NaN
4,5,BUKIT_MERAH,OTHER,IMPROVED,1990-01-01,11.0,21,0.887322,29.0,251.724138,...,1.294327,1.620819,0.431442,0.691163,0.311404,0.448964,1,61.9,448.142165,NaN
5,6,BUKIT_MERAH,OTHER,IMPROVED,1990-01-01,8.0,21,0.887322,29.0,310.344828,...,1.294327,1.620819,0.431442,0.691163,0.311404,0.448964,1,61.9,552.504039,NaN
6,7,BUKIT_MERAH,3_ROOM,IMPROVED,1990-01-01,8.0,7,0.918820,64.0,687.500000,...,1.018641,0.485058,0.986821,0.800866,0.808590,0.940111,1,61.9,1223.949919,NaN
7,8,BUKIT_MERAH,3_ROOM,MODEL_A,1990-01-01,8.0,6,0.920652,74.0,1128.378378,...,1.089870,0.544915,0.567711,0.490547,0.708861,0.799126,1,61.9,2008.841636,NaN
8,9,BUKIT_MERAH,3_ROOM,MODEL_A,1990-01-01,5.0,6,0.920652,73.0,1150.684932,...,1.657478,0.053197,0.982875,0.340834,1.013329,0.940071,1,61.9,2048.553788,NaN
9,10,BUKIT_MERAH,3_ROOM,IMPROVED,1990-01-01,2.0,15,0.902280,76.0,671.052632,...,0.690789,0.083148,0.637859,0.507003,0.406147,1.175060,1,61.9,1194.668821,NaN


,id,actual,pred,period
0,14795,701.492537,696.97970,1991Q2
1,14796,765.625000,639.54364,1991Q2
2,14797,1025.641026,1014.72345,1991Q2
3,14798,755.319149,743.48960,1991Q2
4,14799,697.014925,667.50010,1991Q2
5,14800,677.966102,544.91565,1991Q2
6,14801,880.597015,710.52576,1991Q2
7,14802,712.328767,828.21450,1991Q2
8,14803,694.117647,658.36650,1991Q2
9,14804,377.049180,422.94037,1991Q2


In [57]:
df = pred_df.merge(original_df, how = "inner", on="id")
display(df.head(5))
display(df.columns)

,id,actual,pred,period,town,flat_type,flat_model,sale_date,floor,age,...,nearest_post_offices,nearest_bus_stops,nearest_pharmacies,nearest_primary_schools,nearest_parks,nearest_trains,sale_month,cpi,cpi_psm,avg_sora
0,14795,701.492537,696.97970,1991Q2,CLEMENTI,3_ROOM,NEW_GENERATION,1991-04-01,5.0,13,...,0.445320,0.149947,0.522394,0.409023,0.805405,0.481997,16,64.6,1196.663740,NaN
1,14796,765.625000,639.54364,1991Q2,BUKIT_BATOK,3_ROOM,SIMPLIFIED,1991-04-01,5.0,4,...,1.255801,0.052717,0.975715,0.571979,0.594545,1.187488,16,64.6,1306.066176,NaN
2,14797,1025.641026,1014.72345,1991Q2,BEDOK,5_ROOM,STANDARD,1991-04-01,11.0,13,...,0.873955,0.154340,1.138649,0.270174,0.404171,0.899955,16,64.6,1749.622926,NaN
3,14798,755.319149,743.48960,1991Q2,JURONG_EAST,4_ROOM,NEW_GENERATION,1991-04-01,2.0,8,...,0.588043,0.128560,0.552882,0.459356,1.025006,0.663620,16,64.6,1288.485607,NaN
4,14799,697.014925,667.50010,1991Q2,JURONG_EAST,3_ROOM,NEW_GENERATION,1991-04-01,5.0,8,...,0.403156,0.136998,0.285124,0.581021,1.289216,0.515997,16,64.6,1189.025461,NaN


Index(['id', 'actual', 'pred', 'period', 'town', 'flat_type', 'flat_model',
       'sale_date', 'floor', 'age', 'relative_tenure', 'floor_area', 'psm',
       'address', 'latitude', 'longitude', 'latitude_rad', 'longitude_rad',
       'nearest_atm', 'nearest_libraries', 'nearest_hawkers', 'nearest_stores',
       'nearest_post_offices', 'nearest_bus_stops', 'nearest_pharmacies',
       'nearest_primary_schools', 'nearest_parks', 'nearest_trains',
       'sale_month', 'cpi', 'cpi_psm', 'avg_sora'],
      dtype='object')

### Grouped by latitude and longitude

In [76]:
def produce_grouped_coords(df):
  new_df = deepcopy(df)
  new_df = new_df[(103 <= new_df["longitude"]) & (new_df["longitude"] <= 104.5) &
                  (1 <= new_df["latitude"]) & (new_df["latitude"] < 2)]
  new_df = new_df.assign(abs_err = lambda x: np.abs(x["pred"] - x["actual"])*100/x["actual"],
              combined_coords = lambda x: x["longitude"].astype(str) + ", " + x["latitude"].astype(str))
  res_df = new_df.groupby(by="combined_coords")["abs_err"].agg("mean").to_frame()
  return res_df

In [87]:
res_df = produce_grouped_coords(df).reset_index()
res_df[["longitude", "latitude"]] = res_df["combined_coords"].str.split(", ", expand=True)
res_df[["longitude", "latitude"]] = res_df[["longitude", "latitude"]].astype(np.float32)
res_df = res_df.drop(columns="combined_coords")
display(res_df.head(10))

,abs_err,longitude,latitude
0,8.897552,103.685165,1.340518
1,6.768591,103.685524,1.341542
2,6.563180,103.685982,1.340221
3,6.063782,103.685982,1.341833
4,5.755638,103.686005,1.341082
5,6.364253,103.686066,1.339488
6,6.382602,103.686249,1.341318
7,7.272533,103.686432,1.340511
8,6.448952,103.686569,1.339107
9,6.099785,103.686882,1.341901


In [98]:
fig = px.scatter_mapbox(res_df, lat="latitude", lon="longitude", hover_name="abs_err", color = "abs_err",
                  color_continuous_scale = px.colors.cyclical.IceFire,
                  width = 1000, height = 800).update_layout(
  mapbox={
    "style": "carto-positron",
    "zoom": 10,
  }
)
fig.write_html("./v1-map.html")